In [1]:
# Importing modules
import pandas as pd
import wget
import csv
import random
import os
from bs4 import BeautifulSoup
from requests import get
import re
import sqlite3
from tkinter import *
import webbrowser

In [2]:
build = "1.01 30.06.2021"

In [3]:
# Creating tables

woman_names = []
man_names = []

woman_surnames = []
man_surnames = []


try:

# Downloading female names from  https://dane.gov.pl/pl/dataset/1501,lista-imion-wystepujacych-w-rejestrze-pesel/resource/28099/table?page=1&per_page=20&q=&sort=    
    
    woman_names_url = r'https://bit.ly/3iXOZzs'
    woman_names_file = wget.download(woman_names_url)


    with open(woman_names_file, 'rt',encoding='utf-8' ,errors="ignore") as woman_names_csv:
        
        csv_read_woman = csv.DictReader(woman_names_csv)
        
        for row in csv_read_woman:
                woman_names.append(row['IMIĘ_PIERWSZE'])
                
    os.remove('3iXOZzs')

            
# Downloading male names from https://dane.gov.pl/pl/dataset/1501,lista-imion-wystepujacych-w-rejestrze-pesel/resource/28100/table?page=1&per_page=20&q=&sort=


    man_names_url = r'https://bit.ly/3qpdUgX'
    man_names_file = wget.download(man_names_url)


    with open(man_names_file, 'rt',encoding='utf-8' ,errors="ignore") as man_names_csv:
        
        csv_read_man = csv.DictReader(man_names_csv)
        
        for row in csv_read_man:
                man_names.append(row['IMIĘ_PIERWSZE'])
                
    os.remove('3qpdUgX')
            
# Downloading woman surnames from https://dane.gov.pl/pl/dataset/1681,nazwiska-osob-zyjacych-wystepujace-w-rejestrze-pesel/resource/28080/table?page=1&per_page=20&q=&sort=

    woman_surnames_url = r'https://bit.ly/3zMpBCK'
    woman_surnames_file = wget.download(woman_surnames_url)


    with open(woman_surnames_file, 'rt',encoding='utf-8' ,errors="ignore") as woman_surnames_csv:
        
        csv_read_woman_surnames = csv.DictReader(woman_surnames_csv)
        
        for row in csv_read_woman_surnames:
                woman_surnames.append(row['Nazwisko aktualne'])
                
    os.remove('3zMpBCK')

# Downloading man surnames from https://dane.gov.pl/pl/dataset/1681,nazwiska-osob-zyjacych-wystepujace-w-rejestrze-pesel/resource/28052/table?page=1&per_page=20&q=&sort=             

    man_surnames_url = r'https://bit.ly/3japaMq'
    man_surnames_file = wget.download(man_surnames_url)

    with open(man_surnames_file, 'rt',encoding='utf-8' ,errors="ignore") as man_surnames_csv:
        
        csv_read_man_surnames = csv.DictReader(man_surnames_csv)
        
        for row in csv_read_man_surnames:
                man_surnames.append(row['Nazwisko aktualne'])
                
    os.remove('3japaMq')
                
except ValueError:
    print('Files already exist!')
    pass


except KeyError:
    print('Wrong dictionary key')

In [4]:
def rand_consumer_list(num=1, man_possibility=1.0, woman_possibility=1.0, min_age=18, max_age=70):
    
    global consumer_column
    consumer_column = ['Imię','Nazwisko','Wiek','Płeć']
    
    global consumer_list
    consumer_list = []
    x = 0
    
    while x < num:
        possibility_roll = random.random()
        
        if possibility_roll <= man_possibility:
            
            consumer_list.append([random.choice(man_names),
                                  random.choice(man_surnames),
                                  random.randint(min_age,max_age),
                                  "MĘŻCZYZNA"
                                 ])
                
                
        if possibility_roll <= woman_possibility:
            
            consumer_list.append([random.choice(woman_names),
                                  random.choice(woman_surnames),
                                  random.randint(min_age,max_age),
                                  "KOBIETA"
                                 ])
        x+=1

In [5]:
food_column_names = ['Produkt', 'Cena', 'Cena za jednsotkę', 'Jednostka']
food_list = []


def price_format(price):
    return float(price.replace(" ", "").replace("zł", "").replace(",", "."))


def pname_format(product):
    return product.replace(",", ".")


def food_data(last_page_num=1):
    url = 'https://www.carrefour.pl/artykuly-spozywcze'

    for page_num in range(0, last_page_num):

        site = get(f'{url}?page={page_num}')
        bs = BeautifulSoup(site.content, 'html.parser')

        for offer in bs.find_all('div', class_='jss304'):

            try:

                product = pname_format(
                    offer.find('span', class_='MuiTypography-root jss199 jss310 MuiTypography-h5').get_text().strip())

                price = price_format(
                    offer.find('div', class_='MuiTypography-root jss318 MuiTypography-h3').get_text().strip())

                measure_html = offer.find('span',
                                     class_='MuiTypography-root jss199 jss321 MuiTypography-caption MuiTypography-colorTextSecondary').get_text().strip()

                if re.search(r'kg\Z', measure_html):
                    measure = "Kilogramy"

                elif re.search(r'szt.\Z', measure_html):
                    measure = "Sztuki"

                elif re.search(r'l\Z', measure_html):
                    measure = "Litry"

                price_kg_raw = offer.find('span',
                                          class_='MuiTypography-root MuiTypography-caption MuiTypography-colorTextSecondary').get_text().strip()

                if re.search(r'zł/1 kg\Z', price_kg_raw):
                    if re.search(r'\A.\xa0', price_kg_raw):
                        price_kg = float(
                            "".join(re.split(r'\xa0', re.split(r'zł/1 kg\Z', price_kg_raw)[0].replace(",", "."))))
                    else:
                        price_kg = float(re.split(r'zł/1 kg\Z', price_kg_raw)[0].replace(",", "."))

                elif re.search(r'szt.\Z', price_kg_raw):
                    price_kg = float(re.split(r' zł/1 szt.\Z', price_kg_raw)[0].replace(",", "."))

                elif re.search(r'l\Z', price_kg_raw):
                    price_kg = float(re.split(r' zł/1 l\Z', price_kg_raw)[0].replace(",", "."))

                food_list.append([product, price, price_kg, measure])

            except AttributeError:
                pass


def data_to_csv(csv_file_name='products', product_list=food_list, column_names=food_column_names):
    with open(f'{csv_file_name}.csv', 'w', encoding='utf-8', newline='') as products_csv:
        writer = csv.writer(products_csv)
        writer.writerow(column_names)
        writer.writerows(product_list)


def data_to_DB(DB_name='products', table_name='Products', product_list=food_list, truncate=True,  column_names=food_column_names):
    con = sqlite3.connect(f'{DB_name}.db', timeout=15)

    cur = con.cursor()

    try:
        cur.execute(f'CREATE TABLE {table_name}{tuple(column_names)}')
    except:
        if truncate:
            cur.execute(f'DROP TABLE {table_name};')
            cur.execute(f'CREATE TABLE {table_name}{tuple(column_names)}')
        pass

    for product in product_list:
        cur.execute(f'INSERT INTO {table_name} VALUES {tuple(product)}')

    con.commit()
    con.close()


In [6]:
def github_click():
    webbrowser.open('https://github.com/4rturF')

In [ ]:
app = Tk()
app.title('Data Randomizer 1.0')
app.geometry('900x700')
app.resizable(0,0)
try:
    app.iconbitmap('C:/Users/artur/Randomizer/bin/icon.ico')
except:
    pass

logo_frame = LabelFrame(app, bd=1, bg='#66cc66')
logo_frame.place(x=0,width=300, height=630)

logo = Label(logo_frame, text='Data\nRandomizer', font=("Agency FB", 48, 'bold','italic'), bg='#66cc66',fg="#404040", justify=LEFT)
logo.place(x=0, y=10)

build_info = Label(logo_frame, text=f'Author: Artur Fratczak \nBuild: {build}',font=("Agency FB", 12, 'bold'), bg='#66cc66',fg="#404040", justify=LEFT )
build_info.place(x=0, y=580)


wizard_frame = LabelFrame(app,bd=0, bg='#d9d9d9')
wizard_frame.place(x=300,width=600,height=100)

buttons_frame = LabelFrame(app,relief=SUNKEN, bd=1, bg='#e6e6e6')
buttons_frame.place(y=630, width=900,height=70 )

exit_button = Button(buttons_frame, text="Zakończ", width=12, command=app.destroy)
exit_button.pack(side=RIGHT, padx=10)

next_button = Button(buttons_frame, text="Dalej", width=10, font="bold")
next_button.pack(side=RIGHT, padx=40)

github_button = Button(buttons_frame, text="Github", width=10, command=github_click)
github_button.pack(side=LEFT, padx=30)


app.mainloop()